# Compaction Solver

This notebook solves the viscous compaction problem in one spatial dimension (vertical). 

For a brief overline of the problem and numerical method, see the notebook notes.ipynb.

The code requires [FEniCSx](https://fenicsproject.org). The notebook can be run 
through a [Docker](https://www.docker.com) container with the command:

`docker run --init -ti -p 8888:8888 -v $(pwd):/home/fenics/shared -w /home/fenics/shared dolfinx/lab:stable`

Various imports:

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

Set the main dimensional parameters in the problem:            
(**note:** to modify these paramters after running the cell, you have to restart the kernel)

In [ ]:
import meta_params as mp
mp.H0 = 0.018                                    # initial column height (m)
mp.w0 = 111/3.154e7                              # prescribed compaction rate (m/s)
mp.phi0 = 0.65                                   # initial porosity 
mp.eta = 5e7                                     # Newtonian viscosity (Pa s)
mp.k0 = 1e-10                                    # permeability pre-factor (m^2)
mp.t_f = 25*60                                   # Final time (s)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from dolfinx.fem import Constant, Function, FunctionSpace
from dolfinx.mesh import create_interval
from misc import get_stress, interp
from mpi4py import MPI
from params import eta, H0, nt, nz, phi0, w0, zeta, delta
from petsc4py import PETSc
from solvers import full_solve, vel_solve
from ufl import FiniteElement, MixedElement

Define domain:

In [ ]:
# generate mesh, initial domain is scaled height of 1 
domain = create_interval(MPI.COMM_WORLD,nz,[0,1])

Define initial porosity:

In [ ]:
P1 = FiniteElement('P',domain.ufl_cell(),1)     
element = P1*P1
V = FunctionSpace(domain,element)   
initial = Function(V)
initial.sub(1).interpolate(lambda x:phi0+0*x[0])

Define the boundary conditions (stress or velocity) at the top of the domain:

In [ ]:
# # Dirichlet condition: set compaction rate (velocity) at the top
# bc_top = {'type': 'velocity', 'value': -1}

# # Neumann condition: set the load (stress) at the top
stress = 1e5 # units: Pa
stress_scale = w0*((4./3.)*eta + zeta)/H0 
bc_top = {'type': 'stress', 'value': stress/stress_scale}

Solve the momentum balance for the initial porosity to obtain the initial velocity:

In [ ]:
w_i = vel_solve(domain,phi0,bc_top)
initial.sub(0).interpolate(w_i)

Solve the full problem over all time steps:

In [ ]:
w,phi,sigma,z = full_solve(domain,initial,bc_top)

Plot the solution:

In [ ]:
# time array for plotting
t = np.outer(np.linspace(0,1,nt),np.ones(nz+1))
sigma0 = np.abs(sigma[0,-1])
d = int(np.abs(sigma).max()/sigma0)

plt.figure(figsize=(8,4))
plt.subplot(131)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,-w,cmap='Blues',levels=np.linspace(0,1,100),extend='both')
plt.ylabel(r'$z$',fontsize=18)
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar.set_label(r'$w\,/\,w_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])

plt.subplot(133)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,np.abs(sigma)/sigma0,cmap='Blues',levels=np.linspace(0,d,100),extend='both')
plt.xlabel(r'$t$',fontsize=18)
plt.ylabel(r'$z$',fontsize=18)
plt.xticks(fontsize=12)
plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,d,5))
cbar.set_label(r'$\Sigma\,/\,\Sigma_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)

plt.subplot(132)
plt.plot(t[:,0],z[:,-1],'--',color='crimson',linewidth=1)
plt.contourf(t,z,phi/phi0,cmap='Blues',levels=np.linspace(0,1,100),extend='both')
plt.xlabel(r'$t$',fontsize=18)
plt.xticks(fontsize=12)
plt.gca().yaxis.set_ticks([])
plt.yticks(fontsize=12)
cbar = plt.colorbar(orientation='horizontal',pad=0.2,ticks=np.linspace(0,1,5))
cbar.set_label(r'$\phi\,/\,\phi_0$',fontsize=24)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_xticklabels([0,0.25,0.5,0.75,1])
plt.tight_layout()
plt.show()
plt.close()

Create a stress-displacement plot (dimensional):

In [ ]:
# # import data for comparison if available
# fname = 'filename.csv'
# data = np.genfromtxt(fname, delimiter=',')
# dH_data = data[:,0]
# sigma_data = data[:,2]

dH = -(z[:,-1] - z[0,-1])
sigma_H = -(sigma[:,-1]-sigma[0,-1]) #note(!): we have to subtract initial stress.... ?!?!?
sigma_sc = (w0/H0)*((4./3.)*eta + zeta)/1e3
dH_sc = H0*1e3

plt.figure(figsize=(8,6))
plt.plot(dH*dH_sc,sigma_H*sigma_sc,color='royalblue',linewidth=3)
# plt.plot(dH_data,sigma_data,color='crimson',marker='o',markersize=8)
plt.ylabel(r'$\Sigma-\Sigma_0$ (kPa)',fontsize=20)
plt.xlabel(r'$\delta$ (mm)',fontsize=20)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlim(0,5)
plt.ylim(-10,150)
plt.show()
plt.close()